In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

file_dir = './corpus_6_4000'

file_list = os.listdir(file_dir)
file_list[:5]

['Auto_0.txt', 'Auto_1.txt', 'Auto_10.txt', 'Auto_100.txt', 'Auto_1000.txt']

In [2]:
len(file_list)

24000

In [3]:
datas = []
classes = []
for file in file_list:
    file_path = file_dir + '/' + file
    with open(file_path) as f:
        data = f.read()
    datas.append(data)
    cla = file.split('_')[0]
    classes.append(cla)

In [4]:
datas[:3]

['大C4毕加索首现优惠 现车降5000元现货供应\n网上车市1月4日报道 \n是以进口方式引进的一款集合了多项科技的MPV轿车。它搭载了雪铁龙最新的技术，将中央集控式方向盘、全新CVVT可变气门发动机引入其中，根据网上车市价格监测系统显示，目前大C4毕加索最高可优惠现金5000元。\n据4S店销售人员介绍，目前C4大毕加索现车可优惠现金5000元，全系车型有舒适版和豪华版两款车型，售价分别为27.68万和30.68万。目前现车供货比较充足，现车颜色也比较齐全。消费者可在元旦期间到店看车选购。\n \n大C4毕加索给人的第一感觉就是“大”：车长4.59米、轴距超过了2.7米，车顶高1.68米；超大的前挡风玻璃几乎延伸到了前排乘客的头顶，可视角度达到了惊人的70度，为驾驶者提供了一个非常出色的视野。复合式设计的超大头灯摄人心魂；前散热格栅上宽大的镀铬双人字齿轮标一直延伸到两个前车灯内侧，为大C4毕加索更添了一分干练和强悍；采用的柱式LED组合的大面积直立式尾灯十分醒目，同时也提高了夜间行车的辨识性。\n \n大C4-毕加索的内部整理箱可谓是一应俱全，除了常规的手套箱之外，正、副驾驶前面的中控台上各有一个开启式的储物盒。另外大C4毕加索放弃了传统的换档操控位置，将四档自动电子变速箱的换档杆和换档拨片都集中到方向盘上，从而使前排中部的车内空间更宽裕，驾驶操作也更便捷，而且还得到了一个额外的空间奖励：一个位于中控台下方的5.4升大容积冷藏箱。\n\n',
 '海马广州隆润丘比特上市发布 当天订单达32台\n网易汽车1月4日报道\n2009年12月23日\n，海马汽车宣布，旗下首款A0级家轿海马丘比特正式上市。\n2010年1月1日\n，虎年第一天，海马汽车广州隆润销售服务店在琶洲保利展馆的“2010广州新春汽车展”上隆重举行了“丘比特上市发布会”，在发布当天订单达32台。\n定位于新一代"高·品·值"家轿的丘比特，此次共推出\n1.3L\n、\n1.5L\n两种排量四款车型，价格分别为\n1.3L\n手动实用型5.39万元，\n1.3L\n手动舒适型5.89万元，\n1.3L\n手动豪华型6.29万元，\n1.5L\n手动豪华型6.59万元，共有拉丁黄、光辉红、冰河蓝、时尚黑、动感蓝、星月银、极地白七种颜色可供选择。丘比特象征着爱，丘比特为爱而生。它将承载着所有用户对家庭、

In [16]:
set(classes)

{'Auto', 'Culture', 'Economy', 'Medicine', 'Military', 'Sports'}

In [6]:
import jieba
import re
stop_word = []
with open('stopwords_cn.txt','r') as f:
    for word in f.readlines():
        stop_word.append(word.strip())
stop_word.append('\u3000')

In [7]:
def make_word_dict(training_data):
    word_dict = {}
    for data in training_data:
        data_list = jieba.cut(re.sub('[a-zA-Z0-9]','',data.strip()))
        for word in data_list:
            if word not in stop_word and len(word)>1:
                if word in word_dict.keys():
                    word_dict[word] += 1
                else:
                    word_dict[word] = 1
    word_freq = sorted(word_dict.items(), key=lambda f:f[1], reverse=True)
    return word_dict,word_freq

In [8]:
word_dict,word_freq = make_word_dict(datas)

word_freq[:30]

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/s5/cxqfsb816f36h8gp706x5kjm0000gn/T/jieba.cache
Loading model cost 0.961 seconds.
Prefix dict has been built succesfully.


[('中国', 43812),
 ('一个', 29356),
 ('市场', 23137),
 ('汽车', 22275),
 ('没有', 20719),
 ('已经', 17960),
 ('发展', 17408),
 ('进行', 16574),
 ('目前', 15792),
 ('公司', 15480),
 ('问题', 15072),
 ('表示', 14901),
 ('记者', 14688),
 ('文化', 14311),
 ('可能', 13462),
 ('工作', 13163),
 ('国家', 13156),
 ('北京', 12984),
 ('网易', 12710),
 ('认为', 11984),
 ('.%', 11888),
 ('美国', 11732),
 ('比赛', 11615),
 ('经济', 11422),
 ('成为', 11016),
 ('企业', 10861),
 ('方面', 10685),
 ('车型', 10465),
 ('现在', 10403),
 ('医院', 10308)]

In [9]:
#构造特征
def make_feature(topN,word_freq):
    feature_words = [x[0] for x in word_freq[:topN]]
    return feature_words

#构造文本的特征
def make_text_feature(text,feature_words):
    text_word_dict,_ = make_word_dict(text)
    text_feature = np.zeros(len(feature_words))
    
    for i in range(len(feature_words)):
        if feature_words[i] in text_word_dict.keys():
            text_feature[i] = 1
    return text_feature

import random
import numpy as np
import pickle
import os
#构造训练集、验证集、测试集
def make_data_set(datas,classes,topN,word_freq):
    if os.path.exists('dataset%d.pkl'%topN):
        data_set = pickle.load(open('dataset%d.pkl'%topN,'rb'))
    else:
        data_set = []
        feature_words = make_feature(topN,word_freq)
        for data in datas:
            text_feature = make_text_feature(data,feature_words)
            data_set.append(text_feature)
        pickle.dump(data_set,open('dataset%d.pkl'%topN,'wb'))
    
    data_class_list = list(zip(data_set, classes))
    random.shuffle(data_class_list)
    test_index = int(len(data_class_list)*0.2)+1
    val_index = int(len(data_class_list)*0.3)+1
    
    train_list = data_class_list[val_index:]
    val_list = data_class_list[test_index:val_index]
    test_list = data_class_list[:test_index]
    
    train_data_list, train_class_list = zip(*train_list)
    val_data_list, val_class_list = zip(*val_list)
    test_data_list, test_class_list = zip(*test_list)
    return np.array(train_data_list), np.array(train_class_list),np.array(val_data_list), \
            np.array(val_class_list),np.array(test_data_list), np.array(test_class_list)

In [19]:
train_x,train_y,val_x,val_y,test_x,text_y = make_data_set(datas,classes,10000,word_freq)


In [18]:
train_y.tolist().count('Auto'),train_y.tolist().count('Culture'),train_y.tolist().count('Economy'),train_y.tolist().count('Medicine'),train_y.tolist().count('Military'),train_y.tolist().count('Sports')

(2793, 2826, 2842, 2752, 2776, 2810)

In [20]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB().fit(train_x, train_y)
val_accuracy = classifier.score(val_x, val_y)
print(val_accuracy)

0.172083333333


In [ ]:
from sklearn.naive_bayes import MultinomialNB
for feature_num in range(15000,30001,3000):
    train_x,train_y,val_x,val_y,test_x,text_y = make_data_set(datas,classes,feature_num,word_freq)
    classifier = MultinomialNB().fit(train_x, train_y)
    val_accuracy = classifier.score(val_x, val_y)
    print('feature numbers= %d, accuracy is: ' % feature_num,val_accuracy)

24000 24000
24000
7201 4801
feature numbers= 15000, accuracy is:  0.16375
